<a href="https://colab.research.google.com/github/Zack1243/Aniyomi-Downloader-through-links-file/blob/main/Aniyomi_local_anime_downloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries

In [2]:
import requests
import json
from google.colab import files
import os
from PIL import Image
import re
import shutil
import subprocess
import sys
!python3 -m pip install -U --pre "yt-dlp[default]"
import yt_dlp
import zipfile

## Filter json file

In [3]:
def filter_details(file, url, episode_file):

  # Get correct format options
  with open(file, 'r') as f:
    data = json.load(f)

    # Get info for details.json file
    transformed_data = {
        "title": data.get("title", ""),
        "author": data.get("uploader", ""),
        "artist": data.get("uploader", "can't re-use uploader I guess..."),
        "description": data.get("description", ""),
        "genre": data.get("tags", []),
        # Trying 3 for the status value to see if artist/uploader works
        "status": "2",
        "url": url,
    }

    # Get details for episode.json file
    episode_data = [{
              "episode_number": 1,
              "name": "Ep 1 - " + data.get("title", ""),
              "date_upload": data.get("upload_date", "")
    }]

    url_pattern = r'https?://\S+|www\.\S+'

    # Replace URLs with an empty string
    transformed_data["description"] = re.sub(url_pattern, '', transformed_data["description"])

    # Rewrite .json files WITH PROPER SYNTAX!!!
    with open(file, 'w') as f:
      json.dump(transformed_data, f, indent=4)

    with open(episode_file, 'w') as f:
      json.dump(episode_data, f, indent=4)

## Extract url title and make directory for it

In [4]:
def downloader(url):
    try:
        # Get the video title
        result = subprocess.run([
            'yt-dlp',
            '--no-playlist',
            '--skip-download',
            '--get-title',
            url
        ], capture_output=True, text=True, check=True)

        # Extract the title from the result
        title = result.stdout.strip()

        # Make path to new directory including video's title
        path = os.path.join(os.getcwd(), title)

        # Check if path exists, else make one
        if not os.path.exists(path):
            os.makedirs(path)

        # Make a list for downloading the video
        subprocess.run([
            'yt-dlp',
            '--convert-thumbnails', 'jpg',
            '--progress',
            '--no-playlist',
            '--write-thumbnail',
            '--embed-thumbnail',
            '--add-metadata',
            '--write-info-json',
            '--remux-video', 'mkv',
            '-f', 'bestvideo[height<=1080]+bestaudio/best[height<=1080]/best',
            '-o', f'{path}/%(title)s.%(ext)s',
            url
        ], check=True)


        # Rename cover image to "cover.jpg"
        if os.path.exists(f'{path}/{title}.jpg'):
          os.rename(f'{path}/{title}.jpg', f'{path}/cover.jpg')
        else:
          print("No cover image found.")

        # Create an empty .nomedia file
        os.chdir(path)
        with open('.nomedia', 'w') as makefile:
          pass
        os.chdir('..')

        # Change info.json file to be "details.json"
        if os.path.exists(f'{path}/{title}.info.json'):
          os.rename(f'{path}/{title}.info.json', f'{path}/details.json')

          # Rewrite details.json to aniyomi style guidelines
          filter_details(f'{path}/details.json', url, f'{path}/episode.json')
        else:
          print("No info file found.")

    except subprocess.CalledProcessError as e:
        print("………………………………………………………………………………………………………")
        print(f"An error occurred: {e}")
        print(f"Failed to download url: {url}")
        print("………………………………………………………………………………………………………")

## Example of aniyomi style .json file


```
{
	"title": "Example Title",
	"author": "Example Author",
	"artist": "Example Artist",
	"description": "Example Description",
	"genre": ["genre 1", "genre 2", "etc"],
	"status": "0",
	"_status values": ["0 = Unknown", "1 = Ongoing", "2 = Completed", "3 = Licensed", "4 = Publishing finished", "5 = Cancelled", "6 = On hiatus"]
}
```


## Zip and download files

In [5]:
def zip_and_download_subdirectories():
    # Get the current working directory
    current_directory = os.getcwd()

    # Get a list of all subdirectories
    subdirectories = [d for d in os.listdir(current_directory) if os.path.isdir(os.path.join(current_directory, d))]

    # Create a zip file
    with zipfile.ZipFile('anizip.zip', 'w') as zip_file:
        # Iterate through each subdirectory
        for subdirectory in subdirectories:
            # Get a list of all files in the subdirectory
            files_in_subdir = [os.path.join(subdirectory, f) for f in os.listdir(os.path.join(current_directory, subdirectory))]

            # Iterate through each file in the subdirectory
            for file_path in files_in_subdir:
                # Add the file to the zip file
                zip_file.write(os.path.join(current_directory, file_path), arcname=file_path)

    # Download the zip file
    files.download('anizip.zip')


## Delete videos/images

In [6]:
def purge_content():
    # Get the current working directory
    current_directory = os.getcwd()

    # Get a list of all subdirectories
    subdirectories = [d for d in os.listdir(current_directory) if os.path.isdir(os.path.join(current_directory, d))]

    # Iterate through each subdirectory
    for subdirectory in subdirectories:
        subdirectory_path = os.path.join(current_directory, subdirectory)
        # Get a list of all files in the subdirectory
        files_in_subdir = [os.path.join(subdirectory_path, f) for f in os.listdir(subdirectory_path)]

        # Iterate through each file in the subdirectory
        for file_path in files_in_subdir:
            # Delete the file
            os.remove(file_path)
        # Delete the subdirectory
        os.rmdir(subdirectory_path)

## Get urls

In [7]:
if 'anizip' not in os.getcwd():
  try:
    os.mkdir(os.path.join(os.getcwd(), 'anizip'))
    os.chdir(os.path.join(os.getcwd(), 'anizip'))
  except:
    os.chdir(os.path.join(os.getcwd(), 'anizip'))

# Get urls from user
urls = input("Enter urls separated by space or enter: ")
urls = urls.split()

# Download each url
for url in urls:
  downloader(url)

# Zip files and download
zip_and_download_subdirectories()

# Get rid of the videos/images
purge_content()

Enter urls separated by space or enter: https://www.youtube.com/watch?v=dFYYImRNR6U


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>